In [217]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import json
import pandas as pd
import warnings
import time 
import platform
import pymysql
import pandas as pd
import random

headers = {'Content-Type':"application/json"
        ,"accept":"*/*",'accept-encoding':'gzip, deflate, br','accept-language':"ko","User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"}

def bus_id_from_db():
    conn = pymysql.connect(host="43.200.202.164",port=3307,user="root",password="qkqh1234!",database="finalproject")

    cur = conn.cursor()

    sql = "SELECT bus_id,restaurant_name FROM restaurant"

    cur.execute(sql)

    data = cur.fetchall()

    conn.close()
    df = pd.DataFrame(data)

    df.to_csv('busId_list.csv',encoding='utf-8-sig')
    

def parsing_review():
    places = pd.read_csv('./busId_list.csv').drop(columns=['Unnamed: 0'])
    url = "https://pcmap-api.place.naver.com/graphql"

    df_list = []
    for bus_id in places.iloc[:,0]:
        try: 
            df_list.append(pd.DataFrame(json.loads(requests.post(headers=headers, url=url, 
            data=json.dumps({
                    "operationName" : "getVisitorReviewPhotosInVisitorReviewTab",
                    "query" : "query getVisitorReviewPhotosInVisitorReviewTab($businessId: String!, $businessType: String, $page: Int, $size: Int, $theme: String, $item: String) {\n  visitorReviews(input: {businessId: $businessId, businessType: $businessType, page: $page, size: $size, theme: $theme, item: $item, isPhotoUsed: true, includeReceiptPhotos: false}) {\n    items {\n      id\n      rating\n      author {\n        id\n        nickname\n        from\n        imageUrl\n        objectId\n        url\n        __typename\n      }\n      body\n      thumbnail\n      media {\n        type\n        thumbnail\n        __typename\n      }\n      tags\n      status\n      visited\n      originType\n      item {\n        name\n        code\n        options\n        __typename\n      }\n      businessName\n      isFollowing\n      visitCount\n      votedKeywords {\n        code\n        iconUrl\n        iconCode\n        displayName\n        __typename\n      }\n      __typename\n    }\n    starDistribution {\n      score\n      count\n      __typename\n    }\n    hideProductSelectBox\n    total\n    __typename\n  }\n}\n",

                    "variables" : {
                        "businessId" : f"{bus_id}",
                        "businessType": "restaurant",
                        "item": "0",
                        "page": 1,
                        "size": 20}
                }
            )).text)['data']['visitorReviews']['items'])[['body','thumbnail','businessName']])

        except:
            df_list.append({
                'body':'',
                'thumbnail':'',
                'businessName':''
            })
            pass 
        time.sleep(0.3)

    pd.concat(df_list).to_csv("./review_data.csv",encoding='utf-8-sig')

def reviewing_bot():
    base_url = "http://localhost:8080"

    headers =  { "Content-Type": "application/json" }
    user_data = pd.read_excel('./create_review.xlsx',sheet_name="user_dummy").loc[:,['email','nickName']].dropna()
    review_data = pd.read_excel('./create_review.xlsx',sheet_name="review_dummy")[['body','thumbnail','businessName','email','nickName','busId']]
    review_data['busId'] = review_data['busId'].apply(lambda x : str(x))

    for _,user in user_data.iterrows():
        
        # # 회원가입 수행
        requests.post(url= base_url + "/user/join", data = json.dumps({
            'nickname' : user['nickName'],
            'email' : user['email'],
            'password' : '12345678'
        }), headers=headers).text

        # 로그인 수행
        headers['Authorization'] = json.loads(requests.post(url= base_url + "/login", data=json.dumps({
            "email" : user['email'],
            "password" : "12345678"})).text)['jwtToken']

        for _, review in review_data[review_data['email'] == user['email']].iterrows():
            res = requests.post(url= base_url + f"/review/{review['busId']}/auth/createReview", headers=headers,
                data = json.dumps({
                    "reviewTitle":review['businessName'] + " reviewd by " + review['nickName'],
                    "reviewContent":review['body'],
                    "reviewImage" : review['thumbnail'],
                    "tag":0,
                    "revisit":random.randint(0,1)
                    }))
            time.sleep(0.2)


In [ ]:
# bus_id_from_db()
# parsing_review()
reviewing_bot()